# Scraping a table from Wikipedia and manipulating it

## Importing relevant libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
wiki_table = pd.read_html(str(table))[0]

## Formatting
- Deleting rows with "Not assigned" Borough
- Sorting rows by Postcode
- Resetting index

In [3]:
wiki_table = wiki_table[wiki_table["Borough"] != "Not assigned"]
wiki_table.sort_values('Postcode', inplace=True)
wiki_table.reset_index(inplace=True)
wiki_table.drop("index", axis=1, inplace=True)

## Replacing Neighbourhood "Not assigned" with Borough

In [4]:
for i in range(0,wiki_table['Neighbourhood'].size-1):
    if wiki_table.iloc[i, 2]=="Not assigned":
        wiki_table.iloc[i, 2]=wiki_table.iloc[i, 1]

## Creating the Final table:
- Aggregating rows with same Postcode
- Concatenating Neighbourhood where needed

In [5]:
Final_table = pd.DataFrame(columns = ['Postcode','Borough','Neighbourhood'])
Final_table.loc[0,:]=wiki_table.loc[0,:]
#k=1
#j=1
for i in range(0,wiki_table["Postcode"].size-1):
    if wiki_table.iloc[i+1,0]==Final_table.iloc[Final_table["Postcode"].size-1,0]:
        Final_table.iloc[Final_table["Postcode"].size-1,2]=Final_table.iloc[Final_table["Postcode"].size-1,2]+", "+wiki_table.iloc[i+1,2]
    else:
        Final_table.loc[Final_table["Postcode"].size,:]=wiki_table.loc[i+1,:]
        

# Saving the file as csv

In [6]:
Final_table.to_csv('Final_table.csv')

In [7]:
Final_table.shape

(103, 3)

# Loading Geospatial data

In [8]:
Lon_Lat = pd.read_csv("Geospatial_Coordinates.csv")
Lon_Lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
Final_table = pd.read_csv("Final_table.csv")
Final_table.head()

In [9]:
Lon_Lat.columns= ['Postcode', 'Latitude', 'Longitude']

In [10]:
Final_table = Final_table.merge(Lon_Lat,on='Postcode')

In [11]:
Final_table.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
